# Experimento 1

In [2]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed CodecBzip2 ───────── v0.8.5
   Installed StructTypes ──────── v1.11.0
   Installed JSON3 ────────────── v1.14.3
   Installed BenchmarkTools ───── v1.6.3
   Installed MutableArithmetics ─ v1.6.7
   Installed MathOptInterface ─── v1.46.0
   Installed JuMP ─────────────── v1.29.2
    Updating `~/.julia/environments/v1.11/Project.toml`
  [4076af6c] + JuMP v1.29.2
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [523fee87] + CodecBzip2 v0.8.5
  [0f8b85d8] + JSON3 v1.14.3
  [4076af6c] + JuMP v1.29.2
  [b8f27783] + MathOptInterface v1.46.0
  [d8a4904e] + MutableArithmetics v1.6.7
  [856f2bd8] + StructTypes v1.11.0
  [9abbd945] + Profile v1.11.0
Precompiling project...
   6106.4 ms  ✓ CodecBzip2
   6895.0 ms  ✓ StructTypes
   5830.9 ms  ✓ BenchmarkTools
  19870.2 ms  ✓ MutableArithmetics
  23430.4 ms  ✓ JSON3
  96668.3 ms  ✓ MathOptInterface
 

In [4]:
using JuMP
using GLPK

# --- Conjuntos ---
personas = [:carlos, :ana, :luis, :sofia, :denis]
tareas = Symbol.("t" .* string.(1:12))

# --- Parámetros ---
esSenior = Dict(:carlos => 1, :ana => 0, :luis => 0, :sofia => 1, :denis => 1)
peso = Dict(
    :t1 => 13, :t2 => 8, :t3 => 13, :t4 => 8, :t5 => 13, :t6 => 5,
    :t7 => 8, :t8 => 8, :t9 => 13, :t10 => 3, :t11 => 2, :t12 => 2
)
M = 20

# --- Modelo ---
model = Model(GLPK.Optimizer)

# --- Variables ---
@variable(model, x[p in personas, t in tareas], Bin)
@variable(model, carga[p in personas] >= 0)
@variable(model, obj)

# --- Restricciones y objetivo ---
@objective(model, Max, sum(peso[t] * x[p,t] for p in personas, t in tareas))

# Cada tarea la realiza una sola persona
@constraint(model, [t in tareas], sum(x[p,t] for p in personas) == 1)

# Carga por persona (suma de pesos de tareas asignadas)
@constraint(model, [p in personas],
    carga[p] == sum(peso[t] * x[p,t] for t in tareas))

# Carga mínima por persona
@constraint(model, [p in personas],
    carga[p] >= 18 * esSenior[p] + 13 * (1 - esSenior[p]))

# Carga total del equipo
@constraint(model, sum(carga[p] for p in personas) >= 80)
@constraint(model, sum(carga[p] for p in personas) <= 96)

# --- Resolver ---
optimize!(model)

# --- Resultados ---
println("\nDistribución de tareas por persona:")
println("Persona       Tarea    Peso    Asignado")
for p in personas, t in tareas
    if value(x[p,t]) > 0.5
        println(rpad(string(p),12), rpad(string(t),8), rpad(string(peso[t]),6), "Sí")
    end
end

println("\nCarga total por persona:")
for p in personas
    println(rpad(string(p),12), round(value(carga[p]), digits=2))
end

println("\nValor objetivo (Total de valor del sprint): ", objective_value(model))



Distribución de tareas por persona:
Persona       Tarea    Peso    Asignado
carlos      t1      13    Sí
carlos      t10     3     Sí
carlos      t12     2     Sí
ana         t4      8     Sí
ana         t7      8     Sí
luis        t6      5     Sí
luis        t9      13    Sí
sofia       t2      8     Sí
sofia       t8      8     Sí
sofia       t11     2     Sí
denis       t3      13    Sí
denis       t5      13    Sí

Carga total por persona:
carlos      18.0
ana         16.0
luis        18.0
sofia       18.0
denis       26.0

Valor objetivo (Total de valor del sprint): 96.0


# Experimento 2

In [5]:
using JuMP
using GLPK

# --- Conjuntos ---
personas = [:carlos, :ana, :luis, :sofia, :denis]
tareas = Symbol.("t" .* string.(1:18))

# --- Parámetros ---
esSenior = Dict(:carlos => 1, :ana => 0, :luis => 0, :sofia => 1, :denis => 1)

peso = Dict(
    :t1 => 13, :t2 => 2, :t3 => 13, :t4 => 5, :t5 => 8, :t6 => 13,
    :t7 => 13, :t8 => 2, :t9 => 3, :t10 => 5, :t11 => 8, :t12 => 13,
    :t13 => 13, :t14 => 2, :t15 => 3, :t16 => 5, :t17 => 8, :t18 => 13
)

M = 30  # solo se deja como referencia, ya no es necesario

# --- Modelo ---
model = Model(GLPK.Optimizer)

# --- Variables ---
@variable(model, x[p in personas, t in tareas], Bin)
@variable(model, carga[p in personas] >= 0)

# --- Objetivo ---
@objective(model, Max, sum(peso[t] * x[p,t] for p in personas, t in tareas))

# --- Restricciones ---

# Cada tarea la realiza exactamente una persona
@constraint(model, [t in tareas], sum(x[p,t] for p in personas) == 1)

# Carga individual: suma de pesos de tareas asignadas
@constraint(model, [p in personas],
    carga[p] == sum(peso[t] * x[p,t] for t in tareas)
)

# Carga mínima por persona (depende del rol)
@constraint(model, [p in personas],
    carga[p] >= 27 * esSenior[p] + 20 * (1 - esSenior[p])
)

# Carga total del equipo
@constraint(model, sum(carga[p] for p in personas) >= 121)
@constraint(model, sum(carga[p] for p in personas) <= 146)

# --- Resolver ---
optimize!(model)

# --- Resultados ---
println("\nDistribución de tareas por persona (Caso B):")
println("Persona       Tarea    Peso    Asignado")
for p in personas, t in tareas
    if value(x[p,t]) > 0.5
        println(rpad(string(p),12), rpad(string(t),8), rpad(string(peso[t]),6), "Sí")
    end
end

println("\nCarga total por persona:")
for p in personas
    println(rpad(string(p),12), round(value(carga[p]), digits=2))
end

println("\nValor objetivo (Total de valor del sprint): ", objective_value(model))



Distribución de tareas por persona (Caso B):
Persona       Tarea    Peso    Asignado
carlos      t2      2     Sí
carlos      t5      8     Sí
carlos      t8      2     Sí
carlos      t15     3     Sí
carlos      t16     5     Sí
carlos      t18     13    Sí
ana         t6      13    Sí
ana         t10     5     Sí
ana         t17     8     Sí
luis        t1      13    Sí
luis        t3      13    Sí
sofia       t4      5     Sí
sofia       t7      13    Sí
sofia       t9      3     Sí
sofia       t11     8     Sí
denis       t12     13    Sí
denis       t13     13    Sí
denis       t14     2     Sí

Carga total por persona:
carlos      33.0
ana         26.0
luis        26.0
sofia       29.0
denis       28.0

Valor objetivo (Total de valor del sprint): 142.0
